# Creating the DataFrame
## Creating a pandas dataframe with only the required features from the sqlite database

In [ ]:
#importing neccessary libraries and openning a sqlite connection
import pandas as pd
import sqlite3
from datetime import datetime

con = sqlite3.connect('twitter_followers.db')
print ('Opened database succesfully')

#creating intial dataframe df
df = pd.read_sql('select * from twitter_data1', con)
print (df.head())
print (df.shape)
print (df.columns)

In [ ]:
#accessing the twitter API
import tweepy
consumer_key = 'tHaeI3Vfg3B5BxDdAEnmb5qHp'
consumer_secret = 'bazSR24b793Th2lByeyc7bA319ZTWnx7CvwMVzVzhuCERQbjTn'

access_token = '1140113387952586752-JUA8pQaeQp6z6inciK8TjuqsxwwxL1'
access_token_secret = 'i4stpoFuZJQNMY8xiM9FNpyT02j70lSyiJhL0yChjG7Jd'

auth = tweepy.OAuthHandler (consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth) 

#getting the list of my friends in twitter
my_friends = api.friends_ids() 
print (len(my_friends))

In [ ]:
#creating an empty list to store values: 
#1 -> if the user i in df is my friend
#0 -> if the user i in df is not my friend
is_friend = []
print(len(is_friend))
u_id = df['user_id'] 
print(len(u_id))

for i in u_id:
    if i in my_friends:
        is_friend.append(1)
    else:
        is_friend.append(0)
    
print(is_friend)
print(len(is_friend))

#adding new column 'is_friend' to dataframe df
df['is_friend'] = is_friend

In [ ]:
#adding info about my_followers, similar to my_friends 
my_followers = api.followers_ids()

am_following = []
print(len(am_following))
print(len(u_id))

for j in u_id:
    if j in my_followers:
        am_following.append(1)
    else:
        am_following.append(0)
print(am_following)
print(len(am_following))
df['am_following'] = am_following

In [ ]:
#function to return the number of days from last status update made by user
def count_days(date):
    status_created_at = date.split('-')
    #print(status_created_at) 
    month = status_created_at[0]
    day = status_created_at[1]
    year = status_created_at[-1]
    #print (month)
    #print(day)
    #print(year)
    concated_date = (' '.join([month, day, year]))
    datetime_object = datetime.strptime(concated_date, '%m %d %Y')
    today = datetime.now()
    diff = today - datetime_object
    diff_days = diff.days
    return diff_days

In [ ]:
#evaluating count_days for every user and adding it as new attribute to df
#This feature shows the count of days from the present date when the last status was made
last_status_date = df['user_laststatus_date']
days_laststatus = []

for k in last_status_date:
   d = count_days(k) 
   days_laststatus.append(d)
   
print(len(days_laststatus))
df['days_laststatus'] = days_laststatus

In [ ]:
#creating a new dataframe containing only those members who are my friends
friends = df[:][df.is_friend == 1]
print(friends.shape)
print(friends.columns)
print(friends.head())

In [ ]:
#exporting the dataframe created as a csv file
export_csv = friends.to_csv (
        r'path_of_file.csv', 
        index = None, 
        header=True) 